# Preparation

## Install Dependencies

In [ ]:
!pip install datasets
!pip install tokenizers
!pip install transformers
!pip install stanza
# -- Initialize Stanza --
import stanza
stanza.download('en')

## Download Datasets

In [ ]:
# -- OPEN-I dataset --
!mkdir -p open_i
!wget -q -N -P open_i https://openi.nlm.nih.gov/imgs/collections/NLMCXR_reports.tgz
!tar zxf open_i/NLMCXR_reports.tgz -C open_i
import glob
num_files = len(glob.glob("open_i/ecgen-radiology/*.xml"))
print(f'Downloaded {num_files} files from the OpenI dataset into "open_i/ecgen-radiology"')

# -- Transcriptions Dataset --
!mkdir -p medical_transcriptions
!gdown --id 1E0hm3r9bwK8cujyIcOjp_y-ZEPt1HBjn -O medical_transcriptions/medical_transcriptions.zip
!unzip -o medical_transcriptions/medical_transcriptions.zip -d medical_transcriptions
print(f'Downloaded medical transriptions dataset')

Downloaded 3955 files from the OpenI dataset into "open_i/ecgen-radiology"
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1E0hm3r9bwK8cujyIcOjp_y-ZEPt1HBjn
To: /content/medical_transcriptions/medical_transcriptions.zip
100% 5.08M/5.08M [00:00<00:00, 10.5MB/s]
Archive:  medical_transcriptions/medical_transcriptions.zip
  inflating: medical_transcriptions/mtsamples.csv  
Downloaded medical transriptions dataset


# Training a Text Classifier

## Step 1 - Load the Medical Transctions Dataset
We first load the Medical Transcriptions dataset as a huggingface dataset. The dataset is split into train and test set, so the returned type is a datasets.DatasetDict, which acts like a dictionary of datasets.Dataset but provides utility functions, e.g. for mapping all datasets of the dict using the map method.
This step is already implemented.

In [ ]:
from typing import List, Tuple
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

def load_transciptions_dataset() -> Tuple[DatasetDict, List[str]]:
    df = pd.read_csv('medical_transcriptions/mtsamples.csv')
    df = df.drop(['Unnamed: 0'],axis=1,)

    counts = df['medical_specialty'].value_counts()
    print(f'Original counts:\n{counts}')
    dropped_specialities = [k for k, v in counts.items() if v < 100]
    for dropped_speciality in dropped_specialities:
      df = df[df['medical_specialty'] != dropped_speciality]
    df.dropna(inplace=True)
    counts = df['medical_specialty'].value_counts()
    print(f'Counts after removing small specialities:\n{counts}')

    df['medical_specialty'] = df['medical_specialty'].astype('category')
    class_names = df['medical_specialty'].cat.categories.tolist()
    print(f'Class names : {class_names}')
    df['medical_specialty'] = df['medical_specialty'].cat.codes

    train, test = train_test_split(df, stratify=df['medical_specialty'], test_size=0.25)
    dataset = DatasetDict({'train': Dataset.from_pandas(train), 'test': Dataset.from_pandas(test)})
    dataset = dataset.remove_columns(['__index_level_0__'])

    return dataset, class_names

In [ ]:
dataset, class_names = load_transciptions_dataset()

Original counts:
medical_specialty
 Surgery                          1103
 Consult - History and Phy.        516
 Cardiovascular / Pulmonary        372
 Orthopedic                        355
 Radiology                         273
 General Medicine                  259
 Gastroenterology                  230
 Neurology                         223
 SOAP / Chart / Progress Notes     166
 Obstetrics / Gynecology           160
 Urology                           158
 Discharge Summary                 108
 ENT - Otolaryngology               98
 Neurosurgery                       94
 Hematology - Oncology              90
 Ophthalmology                      83
 Nephrology                         81
 Emergency Room Reports             75
 Pediatrics - Neonatal              70
 Pain Management                    62
 Psychiatry / Psychology            53
 Office Notes                       51
 Podiatry                           47
 Dermatology                        29
 Cosmetic / Plastic Surgery  

In [ ]:
print('Training samples: ', len(dataset['train']['transcription']))
print('Test samples: ', len(dataset['test']['transcription']))
print('Columns: ', dataset['train'].features)

Training samples:  2315
Test samples:  772
Columns:  {'description': Value(dtype='string', id=None), 'medical_specialty': Value(dtype='int8', id=None), 'sample_name': Value(dtype='string', id=None), 'transcription': Value(dtype='string', id=None), 'keywords': Value(dtype='string', id=None)}


## Step 2 - Compute sentence embeddings using pre-trained language model
Our goal is to use a pre-trained language model (from the huggingface transformers library) to encode sentences into sentence representations. Therefore, each sentence is tokenized and passed to the language model. The resulting contextualized token representations (i.e. the outputs of the last hidden layer of the language model) are then globally pooled to get sentence-level representations. As we use a pre-trained model, no training is involved in this step.

### Specify pre-trained language model
Decide which pre-trained language model you want to use and specify its name here.
You can search for models at the huggingface model hub: https://huggingface.co/models . You can either use standard models, e.g. BERT, or use biomedcial models.

When you decided for a model, copy the name of the model from the URL (removing only https://huggingface.co/) and insert it as model_name here.
For more reference on how to use the tokenizer have a look at https://huggingface.co/docs/transformers/preprocessing.

In [ ]:
model_name = 'google-bert/bert-base-uncased'

In [ ]:
# Load the tokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
print(tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

BertTokenizerFast(name_or_path='google-bert/bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


In [ ]:
# Load the model and inspect it
from transformers import AutoModel
model = AutoModel.from_pretrained(model_name)
print(model)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

### Experiment with tokenizer and language model

In [ ]:
# Tokenize some example text and inspect the results to understand the outputs of the tokenizer
sample_text = dataset['train'][:2]['transcription']  # batch of 2 samples of text
print('example text: ', sample_text)
toknized = tokenizer(sample_text, truncation=True, max_length=512)
print(toknized.keys())
print('input ids: ', toknized['input_ids'])
print('attention mask: ', toknized['attention_mask'])
print('length: ', len(toknized['input_ids'][0]))

example text:  ['PREOPERATIVE DIAGNOSIS: , Herniated nucleus pulposus C5-C6.,POSTOPERATIVE DIAGNOSIS: , Herniated nucleus pulposus C5-C6.,PROCEDURE:,  Anterior cervical discectomy fusion C5-C6 followed by instrumentation C5-C6 with titanium dynamic plating system, Aesculap.  Operating microscope was used for both illumination and magnification.,FIRST ASSISTANT: , Nurse practitioner.,PROCEDURE IN DETAIL: , The patient was placed in supine position.  The neck was prepped and draped in the usual fashion for anterior discectomy and fusion.  An incision was made midline to the anterior body of the sternocleidomastoid at C5-C6 level.  The skin, subcutaneous tissue, and platysma muscle was divided exposing the carotid sheath, which was retracted laterally.  Trachea and esophagus were retracted medially.  After placing the self-retaining retractors with the longus colli muscles having been dissected away from the vertebral bodies at C5 and C6 and confirming our position with intraoperative x-r

In [ ]:
# Feed some example data to understand the outputs of the language model
# pad the batch and convert it to Pytorch
# return_tensors='pt' => return the tokenized values as PyTorch tensors instead of lists
x = tokenizer.pad(toknized, return_tensors='pt')
print(x.keys())
print('input ids: ', x['input_ids'])
print('attention mask: ', x['attention_mask'])
print('shape: ', x['input_ids'].shape)

# Encode the tokenized and padded input
results = model(**x)
print('output shape: ', results.last_hidden_state.shape)  # (batch_size x num_tokens x d_hidden)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
input ids:  tensor([[  101,  3653, 25918,  ...,  2008,  2001,   102],
        [  101,  3653, 25918,  ..., 19470,  1012,   102]])
attention mask:  tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])
shape:  torch.Size([2, 512])
output shape:  torch.Size([2, 512, 768])


### Tokenize the dataset
We now tokenize the whole dataset using the batched map-method.

TODO: Implement the tokenize_batch function using the tokenizer. Note: do not pad the data yet but truncate it to a max length of 512.

In [ ]:
def tokenize_batch(text_batch: List[str]):
  # implement tokenize batch using the tokenizer
  return tokenizer(text_batch, truncation=True, max_length=512, return_tensors=None)

In [ ]:
# Apply tokenize_batch to dataset
tokenized_dataset = dataset.map(lambda examples: tokenize_batch(examples["transcription"]), batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(['sample_name', 'transcription', 'description', 'keywords']).rename_column('medical_specialty', 'labels')
print('Columns after tokenization', list(tokenized_dataset['train'].features.keys()))

Map:   0%|          | 0/2315 [00:00<?, ? examples/s]

Map:   0%|          | 0/772 [00:00<?, ? examples/s]

Columns after tokenization ['labels', 'input_ids', 'token_type_ids', 'attention_mask']


### Token Pooling
Define Pooling functions to compute sentence embeddings from token outputs.

implement three possible pooling functions
- CLS: use the output of the [CLS] token only and ignore the other tokens.
- max/avg: globally max/avg pool over all token outputs, but make sure to ignore padding tokens.

In [ ]:
import torch

def CLS_pool(hidden_state: torch.FloatTensor, attention_mask: torch.BoolTensor):
    """
    Returns only the hidden state of the [CLS] token
    :param hidden_state: (N x M x d_hidden) where N is the batch size and M is the number of tokens
    :param attention_mask: (N x M), True for "real" tokens, False for padding tokens.
    :return (N x d_hidden)
    """
    return hidden_state[:, 0, :]  # Select the hidden state of the [CLS] token (first token)

def max_pool(hidden_state: torch.FloatTensor, attention_mask: torch.BoolTensor):
    """
    Globally pools the hidden states over all tokens using max pooling.
    :param hidden_state: (N x M x d_hidden) where N is the batch size and M is the number of tokens
    :param attention_mask: (N x M), True for "real" tokens, False for padding tokens.
    :return (N x d_hidden)
    """
    expanded_mask = attention_mask.unsqueeze(-1).expand_as(hidden_state)  # Expand mask for all d_hidden dimensions
    masked_hidden_state = hidden_state.where(expanded_mask, torch.tensor(float('-inf')).to(hidden_state.device))
    max_pooled = torch.max(masked_hidden_state, dim=1).values
    return max_pooled

def avg_pool(hidden_state: torch.FloatTensor, attention_mask: torch.BoolTensor):
    """
    Globally pools the hidden states over all tokens using average pooling.
    :param hidden_state: (N x M x d_hidden) where N is the batch size and M is the number of tokens
    :param attention_mask: (N x M), True for "real" tokens, False for padding tokens.
    :return (N x d_hidden)
    """
    expanded_mask = attention_mask.unsqueeze(-1).expand_as(hidden_state)  # Expand mask for all d_hidden dimensions
    masked_hidden_state = hidden_state * expanded_mask
    sum_pooled = torch.sum(masked_hidden_state, dim=1)
    token_counts = expanded_mask.sum(dim=1)  # Avoid division by zero; sum of True values per sample
    avg_pooled = sum_pooled / token_counts.clamp(min=1)  # Use clamp to avoid division by zero
    return avg_pooled


### Sentence Embedder
The sentence embedder takes a tokenized input, uses the language model to compute token representations (i.e. the last hidden_state of the language model) and then uses a pooling function to compute a single sentence representation.

implement the sentence embedder forward method.

In [ ]:
from torch import nn
from typing import Dict

class SentenceEmbedder(nn.Module):
    def __init__(self, model, pool):
        super().__init__()
        self.model = model  # Huggingface language model, e.g., AutoModel.from_pretrained('bert-base-uncased')
        self.pool = pool    # Pooling function (CLS_pool, max_pool, avg_pool)

    def forward(self, x: Dict[str, torch.Tensor]) -> torch.Tensor:
        """
        Forward pass for generating sentence embeddings.
        :param x: dict containing:
            - input_ids: torch.Tensor (N x M)
            - attention_mask: torch.Tensor (N x M)
        :return: Sentence embedding for each sample of shape (N x d_hidden)
        """
        # Use the model to get the last hidden state of the language model
        outputs = self.model(input_ids=x['input_ids'], attention_mask=x['attention_mask'])

        # Apply the pooling function to the last hidden states and the attention mask
        pooled_output = self.pool(outputs.last_hidden_state, x['attention_mask'])

        return pooled_output


now decide which pooling function you want to use

In [ ]:
device = 'cuda:0'
dataset_name = 'encoded_transciptions_avg_pool'
pool = avg_pool
sentence_embedder = SentenceEmbedder(model, pool).to(device=device)

 Now run the sentence embedder.

 The resulting dataset will then contain a sentence_embedding column.

In [ ]:
def embed_sentence(batch):
  model_input = {'input_ids': batch['input_ids'], 'attention_mask': batch['attention_mask']}
  model_input = tokenizer.pad(model_input, return_tensors='pt').to(device=device)

  with torch.no_grad():
    sentence_embeddings = sentence_embedder(model_input)
  return {'sentence_embedding': sentence_embeddings.detach().cpu().numpy()}

encoded_dataset = tokenized_dataset.map(embed_sentence, batched=True, batch_size=64)
encoded_dataset.save_to_disk(dataset_name)

Map:   0%|          | 0/2315 [00:00<?, ? examples/s]

Map:   0%|          | 0/772 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2315 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/772 [00:00<?, ? examples/s]

## Step 5 - Train Classification Model on Sentence Embeddings
Now we have a single sentence representation vector for each sentence. We now learn a simple classifier based on a MLP (i.e. a 2-layer fully-connected neural network). We first project each sentence embedding from d_hidden to d_mlp, apply ReLU (or another non-linearity) and then project the resulting vector into num_classes where we then apply the cross entropy loss. The classification head will then be learned based on the training data. This is the only learned component in our model.


In [ ]:
from torch import nn
import datasets
from datasets import load_from_disk
from torch.utils.data import DataLoader
import torch
from tqdm import tqdm
import torch.nn.functional as F

class ClassificationHead(nn.Module):
    def __init__(self, d_hidden: int, d_mlp: int, num_classes: int):
        super().__init__()
        # Define the layers of the MLP
        self.fc1 = nn.Linear(d_hidden, d_mlp)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(d_mlp, num_classes)

        # Define the loss function - Cross Entropy Loss
        self.loss_function = nn.CrossEntropyLoss()

    def forward(self, x, y_true):
        """
        :param x: Sentence embeddings (N x d_hidden)
        :param y_true: Target classes (N)
        :return: Tuple of predictions and loss
        """
        # First fully-connected layer + ReLU activation
        x = self.relu(self.fc1(x))
        # Second fully-connected layer to get logits
        logits = self.fc2(x)

        # Compute the cross-entropy loss
        loss = self.loss_function(logits, y_true)
        # Compute the predictions (class with highest score)
        y_pred = torch.argmax(logits, dim=1)

        return y_pred, loss


Now train the classification head on the sentence embeddings.
The training is already implemented.

In [ ]:
device = 'cuda:0'
dataset_name = 'encoded_transciptions_avg_pool'
num_epochs = 300
lr = 1e-4
weight_decay = 1e-4
d_hidden = 768  # must match the language model hidden output
d_mlp = 1024
batch_size = 128

In [ ]:
# Now run the training...

print('num classes: ', len(class_names))
encoded_dataset = load_from_disk(dataset_name)
# make sure PyTorch Tensors are returned
encoded_dataset.set_format('pt')

# remove all columns except sentence_embedding and labels
columns_to_remove = set(encoded_dataset['train'].column_names) - {'sentence_embedding', 'labels'}
encoded_dataset = encoded_dataset.remove_columns(columns_to_remove)

train_data_loader = DataLoader(encoded_dataset['train'], batch_size=batch_size, shuffle=True)
test_data_loader = DataLoader(encoded_dataset['test'], batch_size=batch_size, shuffle=False)

classification_model = ClassificationHead(d_hidden=d_hidden,
                                          d_mlp=d_mlp,
                                          num_classes=len(class_names))
classification_model = classification_model.to(device=device)
optimizer = torch.optim.Adam(classification_model.parameters(), lr=lr, weight_decay=weight_decay)

classification_model.train()
print('Training...')
for epoch in range(num_epochs):
  train_loss = []
  for train_batch in tqdm(train_data_loader):
    x = train_batch['sentence_embedding'].to(device=device)
    y_true = train_batch['labels'].to(device=device)

    y_pred, loss = classification_model(x, y_true)
    train_loss.append(loss.item())

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
  print('train loss: ', torch.mean(torch.tensor(train_loss)).item())

classification_model.eval()
print('Testing...')
f1_metric = datasets.load_metric("f1")
with torch.no_grad():
  test_loss = []
  for test_batch in tqdm(test_data_loader):
    x = test_batch['sentence_embedding'].to(device=device)
    y_true = test_batch['labels'].to(device=device)

    y_pred, loss = classification_model(x, y_true)

    f1_metric.add_batch(predictions=y_pred, references=y_true)
    test_loss.append(loss)
print('\ntest loss: ', torch.mean(torch.tensor(test_loss)).item())
print('F1 (Macro): ', f1_metric.compute(average="macro"))

num classes:  12
Training...


100%|██████████| 19/19 [00:00<00:00, 69.95it/s]


train loss:  2.255821943283081


100%|██████████| 19/19 [00:00<00:00, 70.63it/s]


train loss:  2.091909408569336


...

train loss:  0.7457561492919922


100%|██████████| 19/19 [00:00<00:00, 216.91it/s]


train loss:  0.7413837313652039
Testing...


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
100%|██████████| 7/7 [00:00<00:00, 199.49it/s]


test loss:  1.362087368965149
F1 (Macro):  {'f1': 0.2817870712526472}



# train loss:  0.7413837313652039
# test loss:  1.362087368965149
# F1 (Macro):  {'f1': 0.2817870712526472}

#  Generating text from a pre-trained decoder LM



## Step 1: Implement generate method
Implement the generate method that takes a language model and its tokenizer together with a list of text prefixes and outputs a list of generated sentences (one for each prefix). The prefixes should be "autocompleted" by the model.
Use the greedy-search method and process all prefixes as a single batch.


In [ ]:
import torch
from typing import List

def generate(model, tokenizer, prefix: List[str], max_predicted: int) -> List[str]:
  """
  :param model: PreTrainedModel.
  :param tokenizer: PreTrainedTokenizer (https://huggingface.co/docs/transformers/main/main_classes/tokenizer#transformers.PreTrainedTokenizer)
  :prefix: Batch of prefixes to be autocompleted by the language model
  """
  if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id
  pad_token_id = tokenizer.pad_token_id
  bos_token_id = tokenizer.bos_token_id
  eos_token_id = tokenizer.eos_token_id
  device = model.device

  # 1. Tokenize the prefixes and prepare them for input into the language model
  #    Notes
  #    - add the start token (BOS) but not the end token (EOS)
  #    - left! pad them to the maximum length in the batch, i.e. paddding is left to the "real" tokens
  #    - inputs_ids and attention_mask should be computes, and should already be stacked in the batch dim
  #    -> this can all be done by calling the tokenizer! See: https://huggingface.co/docs/transformers/main/en/internal/tokenization_utils#transformers.PreTrainedTokenizerBase.__call__
  tokenizer.padding_side = 'left'
  # TODO: call the tokenizer to prepare the inputs (a dictionary)
  inputs = tokenizer(prefixes, return_tensors='pt', padding=True, truncation=True, add_special_tokens=True)
  inputs = inputs.to(device=device)
  input_ids = inputs['input_ids']
  attention_mask = inputs['attention_mask']

  # Compute position ids based on the attention mask
  # Note: the position ids are the indices of the positions, starting with zero and increasing for each non-padding token
  # The positions ids for padding tokens can take any value
  position_ids: torch.LongTensor =  torch.arange(input_ids.shape[-1], device=device).expand(input_ids.shape)

  # Initialize some variables 
  N = position_ids.shape[0]
  past_key_values = None
  unfinished_sequences = torch.ones(N, dtype=torch.bool, device=device)
  predicted_input_ids = input_ids.clone()

  while True:
    # 2. Predict the next token logits by passing the previous inputs into the language model
    #    Note: already computed steps can be given by past_key_values, other steps are given as input_ids
    outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        position_ids=position_ids,
        past_key_values=past_key_values,
        return_dict=True
    )
    # (N x |V|)
    next_token_logits: torch.LongTensor = outputs.logits[:, -1, :]

    # 3. Select the next predicted tokens by greedy search
    # (N)
    next_tokens: torch.LongTensor = torch.argmax(next_token_logits, dim=-1)


    # 4. For all already finished sentences, replace the next_tokens by <PAD>
    next_tokens.masked_fill_(~unfinished_sequences, pad_token_id)

    # 5. Check which sentences are already finished by checking whether they contain <END>
    # Which sentences have been finished with the predicted token (next_tokens)
    # (N)
    newly_finished = next_tokens == eos_token_id

    # Which sentences, therefore, remain unfinished
    # (N)
    unfinished_sequences &= ~newly_finished

    # 6. Concatenate the next input to predicted_input_ids, attention_mask
    # (N x M_new) where M_new is one longer than the previous M of predicted_input_ids
    predicted_input_ids = torch.cat([predicted_input_ids, next_tokens.unsqueeze(-1)], dim=-1)

    # (N x M_new)
    attention_mask = torch.cat([attention_mask, torch.ones_like(next_tokens).unsqueeze(-1).to(device)], dim=-1)
    position_ids = position_ids.amax(dim=1, keepdim=True) + 1
    input_ids = next_tokens.unsqueeze(dim=1)
    past_key_values = outputs.past_key_values

    # 7. Check if finished 
    if predicted_input_ids.shape[1] >= max_predicted or unfinished_sequences.max() == 0:
      break

  # 8. Convert back to text 
  generated_sentences: List[str] = tokenizer.batch_decode(predicted_input_ids, skip_special_tokens=True)
  return generated_sentences




## Step 2: Load a decoder language model

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

decoder_model_name = "gpt2-medium"

decoder_tokenizer = AutoTokenizer.from_pretrained(decoder_model_name)
decoder_model = AutoModelForCausalLM.from_pretrained(decoder_model_name)
decoder_model = decoder_model.to(device="cuda:0")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

## Step 3: Apply the generate method to some example text

In [ ]:
# You can play around with different texts here
prefix_1 = "This sentences is about"
prefix_2 = "Can you complete this sentence?"
prefixes = [prefix_1, prefix_2]

generated_sentences = generate(decoder_model, decoder_tokenizer, prefixes, max_predicted=50)

for i, (prefix, sent) in enumerate(zip(prefixes, generated_sentences)):
  print(f'Completed sentence {i}: "{prefix}"')
  print("-------------------------------------------------------\n")
  print(sent)
  print("\n=======================================================\n\n")

Completed sentence 0: "This sentences is about"
-------------------------------------------------------

This sentences is about about about about about about about about about about about about about about about about about about about about about about about about about about about about about about about about about about about about about about about about about about about about about



Completed sentence 1: "Can you complete this sentence?"
-------------------------------------------------------

Can you complete this sentence?

"I'm not sure if I can do it."

"I'm sorry, but I'm not sure if I can do it."

"I'm sorry, but I'm not sure if





In [ ]:
# You can play around with different texts here
prefix_1 = "The Parthenon is"
prefix_2 = "Parallel programming can be useful in cases where"
prefix_3 = "Freud is regarded as the father of psychoanalysis."
prefixes = [prefix_1, prefix_2, prefix_3]
# For some reason, the function only succeeds in generating a coherent text for the last (or the first if we have only one prefix).
# The rest repeat either the last word of the prefix or a random one.

generated_sentences = generate(decoder_model, decoder_tokenizer, prefixes, max_predicted=50)

for i, (prefix, sent) in enumerate(zip(prefixes, generated_sentences)):
  print(f'Completed sentence {i+1}: "{prefix}"')
  print("-------------------------------------------------------\n")
  print(sent)
  print("\n=======================================================\n\n")

Completed sentence 1: "The Parthenon is"
-------------------------------------------------------

The Parthenon is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is



Completed sentence 2: "Parallel programming can be useful in cases where"
-------------------------------------------------------

Parallel programming can be useful in cases where he he he he he he he he he he he he he he he he he he he he he he he he he he he he he he he he he he he he he he he he



Completed sentence 3: "Freud is regarded as the father of psychoanalysis"
-------------------------------------------------------

Freud is regarded as the father of psychoanalysis. He was born in 1883 in Vienna, Austria. He studied at the University of Vienna and then at the University of Berlin. He was a member of the German Academy of Sciences and the German





As you can see, the results tend to be quite repetetive.
This is why greedy search is typically not used in practice.
Common alternatives include beam search and sampling from the distribution of next tokens. While this is not part of the mandatory exercise, you can try to implement other generation methods as part of a bonus task and see how it can improve the results.